In [1]:
pip install bpemb

In [2]:
from torch import nn
from bpemb import BPEmb
import csv
from sklearn.model_selection import train_test_split
import torch
import torch.autograd as autograd
from torch.autograd import Variable
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math, copy, time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [5]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [6]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [7]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [8]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [9]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [10]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [11]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [12]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [13]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [14]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [15]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [16]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [17]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [18]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [19]:
def make_model(src_vocab, tgt_vocab, N=5, 
               d_model=512, d_ff=512, h=2, dropout=0.3):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), 3),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [20]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [21]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src.to('cuda'), batch.trg.to('cuda'), 
                            batch.src_mask.to('cuda'), batch.trg_mask.to('cuda'))
        loss = loss_compute(out.to('cuda'), batch.trg_y.to('cuda'), batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [22]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0.0 #0.00000017
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 1000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [23]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.1):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [24]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm  
        if self.opt is not None:
            loss.backward()
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm

In [25]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [26]:
import re
bpemb_en = BPEmb(lang="en", vs=25000,dim=300)
bpemb_sa = BPEmb(lang="sa", vs=25000,dim=300)
sans_list = []
eng_list = []
data = []
with open('allData.csv', encoding = "utf-8") as csvfile:
    spamreader = csv.reader(csvfile)
    for k, i in enumerate(spamreader):
        if k > 0:
          sans = re.sub("[|A-Za-z०।॥१२३४५६७८९!@#$%^&*,/\._\]\[\-`~\\n\\t0-9\}\{\"\']"," ",i[1]).split()
          eng = re.sub("[!@#$%^&*/\_\]\[\-`~\\n\\t0-9\}\{\"\']"," ",i[2]).split()
          if 'BEGIN' in sans:
            sans.remove('BEGIN')
          if 'END' in sans:
            sans.remove('END')
          if 'BEGIN' in eng:
            eng.remove('BEGIN')
          if 'END' in eng:
            eng.remove('END')
          if len(sans) > 3 and len(eng) > 3 and len(sans) < 50 and len(eng) < 50 and sans not in sans_list and eng not in eng_list:
              sans = " ".join(sans)
              eng = " ".join(eng)
              sans = bpemb_sa.encode(sans)
              eng = bpemb_en.encode(eng)
              data.append({'sans': sans, 'eng': eng})
              sans_list.append(sans)
              eng_list.append(eng)

In [27]:
with open('parallel_data_new.txt', encoding = "utf-8") as file:
    for each in file:
      i = each.split('START_')
      sans = re.sub("[|A-Za-z०॥।१२३४५६७८९!@#$%^&*,/\._\]\[\-`~\\n\\t0-9\}\{\"\']"," ",i[0]).split()
      eng = re.sub("[!@#$%^&*/\_\]\[\-`~\\n\\t0-9\}\{\"\']"," ",i[1]).split()
      if 'END' == eng[-1]:
        del eng[-1]
      if len(sans) > 3 and len(eng) > 3 and len(sans) < 50 and len(eng) < 50 and sans not in sans_list and eng not in eng_list:
          sans = " ".join(sans)
          eng = " ".join(eng)
          sans = bpemb_sa.encode(sans)
          eng = bpemb_en.encode(eng)
          data.append({'sans': sans, 'eng': eng})
          sans_list.append(sans)
          eng_list.append(eng)

In [28]:
train_data, test_data = train_test_split(data,test_size=0.1,random_state=123)

In [29]:
sans_vocab = {}
eng_vocab = {}
sans_vocab_inv = {}
eng_vocab_inv = {}
for each in train_data:
    for i in each['sans']:
        if i not in sans_vocab.keys():
            sans_vocab[i] = len(sans_vocab) + 2
            sans_vocab_inv[len(sans_vocab_inv) + 2] = i
            
for each in train_data:
    for i in each['eng']:
        if i not in eng_vocab.keys():
            eng_vocab[i] = len(eng_vocab) + 4
            eng_vocab_inv[len(eng_vocab_inv) + 4] = i
            
sans_vocab["<pad>"] = 0
eng_vocab["<pad>"] = 0
eng_vocab["<bos>"] = 1
eng_vocab["<eos>"] = 2
# sans_vocab["<bos>"] = 1
# sans_vocab["<eos>"] = 2
sans_vocab["<unk>"] = 1
eng_vocab["<unk>"] = 3
sans_vocab_inv[0] = "<pad>"
eng_vocab_inv[0] = "<pad>"
eng_vocab_inv[1] = "<bos>"
eng_vocab_inv[2] = "<eos>"
# sans_vocab_inv[1] = "<bos>"
# sans_vocab_inv[2] = "<eos>"
sans_vocab_inv[1] = "<unk>"
eng_vocab_inv[3] = "<unk>"

print(len(sans_vocab), len(eng_vocab))
src_vocab_len, tgt_vocab_len = len(sans_vocab), len(eng_vocab)

13486 12662


In [30]:
batch_size = 128
batch_size_test = 64
train_data.sort(key= lambda x: len(x['sans']) + len(x['eng']))

In [31]:
TrainBatchList = []
for i in range(0, len(train_data), batch_size):
    temp1_sans = []
    temp1_eng = []
    longest_sans = len(max(train_data[i:i+batch_size], key=lambda x: len(x['sans']))['sans'])
    longest_eng = len(max(train_data[i:i+batch_size],  key=lambda x: len(x['eng']))['eng'])
    for a in train_data[i:i+batch_size]:
        temp2_sans = [sans_vocab.get(x, 1) for x in a['sans']]
        temp2_eng = [eng_vocab.get(x, 3) for x in a['eng']]
        zeros_sans = [0] * (longest_sans - len(temp2_sans))
        zeros_eng = [0] * (longest_eng - len(temp2_eng))
        sans = temp2_sans + zeros_sans
        eng = [1] + temp2_eng + [2] + zeros_eng
        temp1_sans.append(sans)
        temp1_eng.append(eng)
    temp1_sans_tensor = autograd.Variable(torch.tensor(temp1_sans))
    temp1_eng_tensor = autograd.Variable(torch.tensor(temp1_eng))
    temp1_batch = Batch(temp1_sans_tensor, temp1_eng_tensor, 0)
    TrainBatchList.append(temp1_batch)

In [32]:
temp1_eng_tensor[:, :-1].shape

torch.Size([90, 105])

In [33]:
TestBatchList = []
for i in range(0, len(test_data), batch_size_test):
    temp1_sans = []
    temp1_eng = []
    longest_sans = len(max(test_data[i:i+batch_size_test], key=lambda x: len(x['sans']))['sans'])
    longest_eng = len(max(test_data[i:i+batch_size_test],  key=lambda x: len(x['eng']))['eng'])
    for a in test_data[i:i+batch_size_test]:
        temp2_sans = [sans_vocab.get(x, 1) for x in a['sans']]
        temp2_eng = [eng_vocab.get(x, 3) for x in a['eng']]
        zeros_sans = [0] * (longest_sans - len(temp2_sans))
        zeros_eng = [0] * (longest_eng - len(temp2_eng))
        sans = temp2_sans + zeros_sans
        eng = [1] + temp2_eng + [2] + zeros_eng
        temp1_sans.append(sans)
        temp1_eng.append(eng)
    TestBatchList.append(Batch(autograd.Variable(torch.tensor(temp1_sans)), autograd.Variable(torch.tensor(temp1_eng)), 0))

In [34]:
pad_idx = 0
model = make_model(src_vocab_len, tgt_vocab_len, N=5)
model.cuda()
criterion = LabelSmoothing(size=tgt_vocab_len, padding_idx=pad_idx, smoothing=0.5)
criterion.cuda()

model_opt = NoamOpt(model.src_embed[0].d_model, 0.5, 2000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))


# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(torch.optim.Adam(model.parameters(), lr=0.0003, betas=(0.9, 0.999), eps=1e-9), mode='min', 
#                                                        factor=0.7, patience=0, 
#                                                        threshold=0.0001, 
#                                                        threshold_mode='rel', 
#                                                        cooldown=0, min_lr=0.00000001, 
#                                                        eps=1e-08, verbose=True)

# 3 layers params:
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(torch.optim.Adam(model.parameters(), lr=0.0003, betas=(0.9, 0.999), eps=1e-9), mode='min', 
#                                                        factor=0.7, patience=5, 
#                                                        threshold=0.0001, 
#                                                        threshold_mode='rel', 
#                                                        cooldown=0, min_lr=0.00000001, 
#                                                        eps=1e-08, verbose=True)
# model_opt = NoamOpt(model.src_embed[0].d_model, 1, 4000,
#             torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
#                                                        factor=0.1, patience=5, 
#                                                        threshold=0.0001, 
#                                                        threshold_mode='rel', 
#                                                        cooldown=0, min_lr=0, 
#                                                        eps=1e-08, verbose=False))
# model.load_state_dict(torch.load("drive/MyDrive/3e_to_5d.pt"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [35]:
i = 1
val_losses = []
import random
for epoch in range(100):
    print("Epoch # : ", i)

    model.train()
    random.shuffle(TrainBatchList)
    run_epoch(TrainBatchList, model, SimpleLossCompute(model.generator, criterion, opt=model_opt))
    print("LR: ", model_opt.optimizer.param_groups[0]['lr'])
    model.eval()
    loss = run_epoch(TestBatchList, model, SimpleLossCompute(model.generator, criterion))
    # scheduler.step(loss.item())
    val_losses.append(loss)
    if loss <= min(val_losses):
        torch.save(model.state_dict(), f"drive/MyDrive/3e_to_5d_v2.pt")
    i+=1
    print(loss)

Epoch # :  1
Epoch Step: 1 Loss: 4.059016 Tokens per Sec: 12388.751953
Epoch Step: 51 Loss: 3.945779 Tokens per Sec: 12427.128906
Epoch Step: 101 Loss: 3.719106 Tokens per Sec: 11716.477539
Epoch Step: 151 Loss: 3.527482 Tokens per Sec: 11143.618164
Epoch Step: 201 Loss: 3.333411 Tokens per Sec: 11454.285156
Epoch Step: 251 Loss: 3.181878 Tokens per Sec: 12010.693359
LR:  3.2240408957185434e-05
Epoch Step: 1 Loss: 3.119868 Tokens per Sec: 21685.470703
Epoch Step: 51 Loss: 3.142305 Tokens per Sec: 20151.753906
tensor(3.1233)
Epoch # :  2
Epoch Step: 1 Loss: 3.025066 Tokens per Sec: 12404.964844
Epoch Step: 51 Loss: 3.047484 Tokens per Sec: 11760.192383
Epoch Step: 101 Loss: 3.058502 Tokens per Sec: 11791.049805
Epoch Step: 151 Loss: 3.050248 Tokens per Sec: 11593.791016
Epoch Step: 201 Loss: 3.004088 Tokens per Sec: 11668.391602
Epoch Step: 251 Loss: 2.955535 Tokens per Sec: 11797.496094
LR:  6.448081791437087e-05
Epoch Step: 1 Loss: 2.997447 Tokens per Sec: 20535.142578
Epoch Step: 51 

KeyboardInterrupt: ignored

In [36]:
 def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys
        

In [37]:
model.eval()

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=512, bias=True)
          (w_2): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.3, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.3, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
 

In [38]:
def translator(batch_input):
  sc = batch_input.src[:1]
  tg = batch_input.trg[:1][0].to('cpu').detach().numpy()
  src = sc[0].to('cpu').detach().numpy()
  len_tg = len(tg)
  sc_mask = (sc != 0).unsqueeze(-2)
  out_ = greedy_decode(model, sc.to('cuda'), sc_mask.to('cuda'), 
                        max_len=len_tg, start_symbol=1)
  first_values = out_[0].to('cpu').detach().numpy()
  first_words = [eng_vocab_inv[i] for i in first_values]
  gt_words = [eng_vocab_inv[i] for i in tg]
  sa_words = [sans_vocab_inv[i] for i in src]
  gt = []
  for i in range(len(gt_words)):
      gt.append(gt_words[i])
  return gt, first_words, sa_words


In [43]:
ground_truth, translated, sanskrit = translator(TestBatchList[10])
print("Translated: ", bpemb_en.decode(translated))
print("Ground Truth: ", bpemb_en.decode(ground_truth))
print("Sanskrit: ", bpemb_sa.decode(sanskrit))

Translated:  <bos> that kausalya, who was lamenting thus, with great difficulty, spoke to the following words, with great difficulty, who was lamenting, with great difficulty.<eos> and with many kinds of trees.<eos>.<eos>.<eos>.<eos>.<eos>.<eos>.<eos>.
Ground Truth:  <bos> encircled by numerous afflictions, bharata weeping for a long time , rolled over the floor and replied to his mother as follows:<eos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Sanskrit:  स रुदत्या चिरम् कालम् भूमौ विपरिवृत्य च जननीम् प्रत्युवाच इदम् शोकैः बहुभिर् आवृतः<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:


a = TestBatchList[0].trg[:1][0].detach().numpy()

In [ ]:
pip install sacrebleu

In [ ]:
import sacrebleu as bl
with open("output_with_3_5layer_v1_2B.txt", "a", encoding="utf-8") as f:
  bleu = 0
  for each in TestBatchList:
    ground_truth, translated, sanskrit = translator(each)
    bleu_sent = bl.sentence_bleu(hypothesis=bpemb_en.decode(translated), references=[bpemb_en.decode(ground_truth)]).score
    bleu+=bleu_sent
    f.write("Translated: "+ bpemb_en.decode(translated)+"\n"+"Ground Truth: "+
            bpemb_en.decode(ground_truth)+"\n"+"Sanskrit: "+ bpemb_sa.decode(sanskrit)
            + "\n" + "Bleu Score for Sentence Pair: "+str(bleu_sent)+"\n")
  f.write("\n Total Average Bleu Score: "+str(bleu/len(TestBatchList)))
  print(bleu/len(TestBatchList))

In [ ]:
import torchtext
torchtext.data.metrics.bleu_score(translated, ground_truth[:248], max_n=2, weights=[0.1, 0.9])